In [1]:
from catboost import CatBoostClassifier, CatBoostRegressor
from scipy import stats
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import precision_score, roc_auc_score, f1_score, recall_score, precision_recall_curve, auc, average_precision_score, precision_recall_fscore_support
from sklearn.preprocessing import StandardScaler
from fancyimpute import IterativeImputer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import GridSearchCV
import xlsxwriter
from random import randint
random_state = 7656
from preprocessing import stds, stats, cv_preprocessing
from load_data import load_data
import os
from scipy.stats import mode

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
#from tensorflow.keras.layers import Conv2D, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

Using TensorFlow backend.
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users

In [2]:
pcls = ["q6.1_INTRU", "q6.2_DREAM", "q6.3_FLASH", "q6.4_UPSET",
               "q6.5_PHYS", "q6.6_AVTHT", "q6.7_AVSIT", "q6.8_AMNES", "q6.9_DISINT",
               "q6.10_DTACH", "q6.11_NUMB", "q6.12_FUTRE", "q6.13_SLEEP",
               "q6.14_ANGER", "q6.15_CONC", "q6.16_HYPER", "q6.17_STRTL"]
phqs = [f"T1q5.{i}" for i in range(1, 10)]
phqs2 = [f"T1q5.{i}_" for i in range(1, 10)]


df_preprocessed, features, target_feature = load_data()

df_preprocessed, features, target_feature = load_data()
df_preprocessed[phqs2] = 0
df_preprocessed[phqs2] = df_preprocessed[phqs] + 1
df_preprocessed['blop1'] = mode(df_preprocessed[phqs2+pcls], axis=1)[0]
df_preprocessed['blop2'] = mode(df_preprocessed[pcls], axis=1)[0]
df_preprocessed['blop3'] = mode(df_preprocessed[phqs2], axis=1)[0]
features.extend(['blop1', 'blop2', 'blop3'])
#df_preprocessed = df_preprocessed[~df_preprocessed['phq3'].isna()]

secondary_targets = {
     'q6.1_INTRU_target':1,
 'q6.2_DREAM_target':1,
 'q6.3_FLASH_target':1,
 'q6.4_UPSET_target':1,
 'q6.5_PHYS_target':1,
    'PCL_Strict3':0
}

secondary_targets2 = {
     'q6.1_INTRU_target':2,
 'q6.2_DREAM_target':2,
 'q6.3_FLASH_target':2,
 'q6.4_UPSET_target':2,
 'q6.5_PHYS_target':2,

    # 'target_avoidance':5,
 #'target_hyper':7,
  #  'phq3':13
}

for j in range(1):
    X, X_out, Y, y_out = train_test_split(df_preprocessed[features + list(secondary_targets.keys())], df_preprocessed['target_binary_intrusion'],\
                                          test_size=0.1, random_state=random_state,\
                                          stratify=df_preprocessed['target_binary_intrusion'])



    for i in [random_state]:
        x_train, x_test, y_train, y_test = train_test_split(X, Y,  random_state=i, test_size=0.25, stratify=Y)
        cat_y_train = y_train
        y_train = [y_train]
        d =x_test
        c = x_train
        for k in secondary_targets.keys():
            y_train.append(x_train[k].apply(lambda x: int(x > secondary_targets[k])))
        for k in secondary_targets2.keys():
            y_train.append(x_train[k].apply(lambda x: int(x > secondary_targets2[k])))
        
        mew = len(secondary_targets) + len(secondary_targets2)
        
        x_train = x_train[features]
        x_test = x_test[features]
    
        x_train, x_test = cv_preprocessing(x_train, x_test, i)

        visible = Input(shape=(x_train.shape[1],))
   

        x = Dense(10, activation='relu')(visible)
        x = Dropout(0.1)(x)
        
        outputs = [Dense(1, activation='sigmoid')(x)]
        
        
        for i in range(mew):
             outputs.append(Dense(1, activation='sigmoid')(x))


        
        
        
        model = Model(inputs=visible, outputs=outputs)
        model.compile(loss=['binary_crossentropy']+['binary_crossentropy'] * mew,
              optimizer=Adam(), loss_weights = [1] + [1e-2] * mew)
        model.fit(x_train,y_train)# , epochs = 150)#, class_weight=[{0:2, 1:1}]+[{0:2,1:1}] * mew, verbose=0)
        # evaluate the model
        
        
        y_pred_test = np.hstack(model.predict(x_test))[0]

        
                
        model = CatBoostClassifier(verbose=0,  class_weights={0:2, 1:1})
        model.fit(x_train ,cat_y_train, early_stopping_rounds=5)

        y_pred = model.predict_proba(np.hstack([x_test, y_pred_test]))[:,1] * y_pred_test
        
    
        for t in np.arange(0.01,1,0.01):
            print(t,'f1_score', f1_score(y_test.astype(int), y_pred>t))

        
        precision, recall, _ = precision_recall_curve(y_test.astype(int), np.vstack(y_pred_test).max(axis=1))
        avs = average_precision_score(y_test.astype(int), np.vstack(y_pred_test).max(axis=1))
        print(f"average_precision_score = {avs}")

        auc_score = auc(recall, precision)
        print(f"pr_auc = {auc_score}")
        plt.plot(recall, precision)
        plt.show()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
364/364 [==============================] - 2s 5ms/sample - loss: 1.1690 - dense_1_loss: 1.0798 - dense_2_loss: 0.9507 - dense_3_loss: 1.0608 - dense_4_loss: 0.7544 - dense_5_loss: 0.6629 - dense_6_loss: 1.0398 - dense_7_loss: 0.5488 - dense_8_loss: 0.8102 - dense_9_loss: 0.9319 - dense_10_loss: 0.6645 - dense_11_loss: 0.8612 - dense_12_loss: 0.6314


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [21]:
y_train

[630    0.0
 578    0.0
 408    0.0
 535    1.0
 638    0.0
       ... 
 266    1.0
 283    0.0
 219    0.0
 233    0.0
 116    0.0
 Name: target_binary_intrusion, Length: 364, dtype: float64,
 630    1
 578    0
 408    0
 535    1
 638    0
       ..
 266    1
 283    1
 219    0
 233    1
 116    0
 Name: q6.1_INTRU_target, Length: 364, dtype: int64,
 630    0
 578    0
 408    0
 535    1
 638    0
       ..
 266    1
 283    0
 219    0
 233    0
 116    0
 Name: q6.2_DREAM_target, Length: 364, dtype: int64,
 630    0
 578    0
 408    0
 535    1
 638    0
       ..
 266    1
 283    0
 219    0
 233    0
 116    0
 Name: q6.3_FLASH_target, Length: 364, dtype: int64,
 630    0
 578    0
 408    0
 535    1
 638    0
       ..
 266    0
 283    1
 219    0
 233    0
 116    0
 Name: q6.4_UPSET_target, Length: 364, dtype: int64,
 630    0
 578    0
 408    0
 535    0
 638    0
       ..
 266    1
 283    0
 219    0
 233    1
 116    0
 Name: q6.5_PHYS_target, Length: 364, dtype: 

In [6]:
y  = model.predict(x_test)

In [8]:
        precision, recall, _ = precision_recall_curve(y_test.astype(int), y)
        avs = average_precision_score(y_test.astype(int), y)
        print(f"average_precision_score = {avs}")

        auc_score = auc(recall, precision)

average_precision_score = 0.33892114450516087


In [7]:
y

array([1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], dtype=int64)